In [6]:
import os
import glob
import pandas as pd

In [7]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/Snai2Counts'

In [8]:
# Add experiment date here to apply to dataframe
analysis_date = '20181110'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Image,ExptDate,Treatment,Stains,Embryo,Somites,ImageMag,Section
0,CntlSide,25,921.287,36.851,20.311,255,9397.129,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
1,ExptSide,26,977.093,37.581,21.251,255,9583.028,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
0,CntlSide,9,530.646,58.961,22.267,255,15034.962,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb2_8ss_40x...,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb2,8ss,40x,sec2
1,ExptSide,18,596.448,33.136,20.390,255,8449.684,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb2_8ss_40x...,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb2,8ss,40x,sec2
0,CntlSide,14,553.267,39.519,23.079,255,10077.362,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb2_8ss_40x...,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb2,8ss,40x,sec3


In [9]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Treatment', 'Embryo', 'Slice', 'ExptDate'])['Count']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Extract means for Cntl and Expt counts
    counts_cntl = df_treatment.xs('CntlSide', level='Slice')['Count'] 
    counts_expt = df_treatment.xs('ExptSide', level='Slice')['Count']

    # Generate ratios as Expt/Cntl
    counts_ratios = pd.DataFrame(counts_expt / counts_cntl)
    counts_ratios.columns = ['Expt/Cntl Counts']

    # Normalize individual values to mean of control group
    norm_cntl = counts_cntl/(float(counts_cntl.mean()))
    norm_expt = counts_expt/(float(counts_cntl.mean()))

    # Combine processed values into single dataframe and output as csv file
    counts_cntl = pd.DataFrame(counts_cntl)
    counts_cntl.columns = ['Cntl Counts']
    counts_expt = pd.DataFrame(counts_expt)
    counts_expt.columns = ['Expt Counts']
    counts_ratios = pd.DataFrame(counts_ratios)
    counts_ratios.columns = ['Expt/Cntl Counts']
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = ['Norm Cntl Counts']
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = ['Norm Expt Counts']
    results = (pd.concat([counts_cntl, counts_expt, counts_ratios, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    
    # Save out results at CSV file, update file name
    results.to_csv(analysis_date + '_' + treatment + '_Snai2CountResults.csv')